Notebook to run:

```bash
python train_and_eval/segmentation_training_transf.py --config configs/PASTIS24/TSViT-S_fold1.yaml --device 0
```


In [1]:
import os
import sys


In [2]:
if "notebooks" in os.getcwd():
    os.chdir(os.path.join("..", "deepsatmodels"))

os.getcwd()


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/muhammed-gpu/code/Users/Muhammed.Miah/DeepSatModels/deepsatmodels'

In [3]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [4]:
from train_and_eval import segmentation_training_transf


ModuleNotFoundError: No module named 'torch'

In [ ]:
config_file: str = "../configs/PASTIS24/TSViT-S_fold1.yaml"
device_ids: list = [0]

segmentation_training_transf.run(config_file=config_file, device_ids=device_ids)


---

In [ ]:

import os
from dotenv import load_dotenv
from azure.ai.ml import command, Input, Output, UserIdentityConfiguration, MLClient
from azure.ai.ml.entities import Environment, Data

# Handle to the workspace
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


In [3]:
try:
    load_dotenv()
    subscription_id = os.environ.get("SUBSCRIPTION_ID")
    resource_group_name = os.environ.get("RESOURCE_GROUP")
    workspace_name = os.environ.get("WORKSPACE_NAME")
except Exception as e:
    print(e)
    print(
        "No .env file found. Created one with environment variables SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME"
    )


In [4]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()


In [ ]:
print(f"Using AzureML Workspace `{workspace_name}` within the RG `{resource_group_name}` and Subscription `{subscription_id}`")
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)


In [ ]:
print("\nAvailable Environments:")
for env in ml_client.environments.list():
    print(f" - {env.name}:{env.latest_version}")


In [ ]:
print("Latest Jobs:")
for index, job in enumerate(ml_client.jobs.list()):
    print(f" - '{job.display_name}'")
    if index > 5:
        break


In [8]:
env_name = "naturevers_inference_env"
env_version = "2"

env = ml_client.environments.get(
        env_name, version=str(env_version)
    )


In [ ]:
datastore = ml_client.datastores.get("covercropatlas")
print(datastore.name, datastore.type, datastore.account_name, datastore.container_name)

datastore_folder = Input(
    path=datastore.id,
    type=AssetTypes.URI_FOLDER,
    mode=InputOutputModes.MOUNT,
)
display(datastore_folder)


In [10]:
job_args = {
    "inputs": dict(
        datastore_folder=datastore_folder
    ),
    "outputs": dict(),
    "code": "../src",  # location of source code
    "command": """python aml_job/train_segmentation_model.py""",
    "environment": env,
    "compute": "FALLOWV100WESTLOW",
    "experiment_name": "deepsatmodels_segmentation_training",
    "display_name": "DeepSatModels Segmentation Training",
    "description": "Training segmentation model using DeepSatModels repository",
    "identity": UserIdentityConfiguration(),
}
job = command(**job_args)


In [ ]:
# ml_client.create_or_update(job)

print("DONE")


In [ ]:
help('modules')


In [19]:
!conda list | grep timm


In [17]:
import timm


In [ ]:
timm.__version__

In [ ]:

path: str = os.path.join("..", "datasets.yaml")

datasets_yaml = Input(
    path=path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.MOUNT,
)

datasets_yaml
